In [ ]:
# !pip install pypdf3
# !pip install tabula-py
# !pip install camelot-py

import pandas as pd
import numpy as np
# import camelot as camel
from PyPDF3 import PdfFileReader 
from tabula import read_pdf 
from tabulate import tabulate

from my_modules import my_functions as mybib
from my_modules import project_functions as pr


%load_ext autoreload
%autoreload 2


# https://www.youtube.com/watch?v=702lkQbZx50 - for tabula
# https://stackoverflow.com/questions/45457054/tabula-extract-tables-by-area-coordinates - for tabula 

In [ ]:
import os 
import re


In [ ]:
#FIXME: close pdf file when error occurs

def extract_all_pages_from_pdf(path, file_name):
    # open path as binary / rb = read binary -> used for pdf files (https://www.youtube.com/watch?v=OZo2HxoIOtw)
            
    pdf = PdfFileReader(open(path, 'rb'))
    last_page = pdf.getNumPages()
    last_page += 1

    # defining page range for import 
    [i for i in range(0, 15)]
    page_range = [i for i in range(5, last_page)]
    print(page_range)

    # prepare column name filler
    column_names = [i for i in range(0, 14)]
    print(column_names)

    # create empty dataframe
    adress_data = pd.DataFrame(columns = column_names)
    display(adress_data)
    
    # extract district information 
    #TODO: nochmal prüfen und vielleicht optimieren
    district = [re.findall(r'(?<=adr_)(.*?)(?=_\d{4}\.pdf)', file_name)]
    print(district)

    for page in page_range:
        
        import_data = read_pdf(path, pages = page, encoding = 'ISO-8859-1', stream = True, area = [175, 33, 783, 520], guess = True, pandas_options={'header': None})
        table_df = import_data[0]

        
        table_df = import_data[0]
        columns_len = len(table_df.columns)

        if columns_len < 14:
            if table_df.iloc[:, 3].dtype in ['int64', 'int32', 'float64', 'float32']:
                table_df.insert(3, 'm1', np.nan)

            if table_df.iloc[:, 6].dtype in ['int64', 'int32', 'float64', 'float32']:
                table_df.insert(5, 'm2', np.nan)
            
            
            column_length = len(table_df.columns)

            
            if column_length < 14:
                table_df['m3'] = np.nan
                # display(table_df)
            
            
            table_df.columns = column_names
            table_df['14'] = str(district[0][0])
            
            adress_data = pd.concat([adress_data, table_df], ignore_index=True , axis=0)
            
            print(f'page: {page}')
            print(adress_data.shape)
    
    return adress_data, district[0][0]




In [ ]:



def get_adress_data_by_year(year):
    
    
    list_of_files = os.listdir(f'../data/input/berlin_adresses/{year}')
    list_of_files
        
    
    for file_name in list_of_files:
        path = f'../data/input/berlin_adresses/{year}/{file_name}'
        print(path)
        
        dataset, name = extract_all_pages_from_pdf(path, file_name)
        
        dataset.to_csv(f'../data/output/temp_adress_data/{name}-{year}.csv', index = False)
        display(dataset.head(5))
        print(dataset.shape)
    




In [ ]:
# get_adress_data_by_year(2020)

In [ ]:
get_adress_data_by_year(2023)

In [ ]:
# file_name = 'adr_friedrichshain-kreuzberg_2020.pdf'

# district = [re.findall(r'(?<=adr_)(.*?)(?=_\d{4}\.pdf)', file_name)]
# district

In [ ]:
# path = '../data/input/berlin_adresses/2023/adr_friedrichshain-kreuzberg_2023.pdf'

# extract_all_pages_from_pdf(path, 'adr_friedrichshain-kreuzberg_2023.pdf')